In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

[점메추 프로그램]

등록할 데이터:

    음식점 분류(중식, 한식, 일식, 분식, 양식, 동남아 음식, 멕시칸 음식, 샐러드)
    음식점 이름
    주소
    네이버 리뷰 별점(없는 경우도 있음)
    학원에서부터 도보로 걸리는 시간
    별도 테이블
    음식점 이름에 따른 대표 메뉴(금액 포함) 리스트
기능:

    음식점 추가
    랜덤 매칭(분류를 정하거나, 분류까지 포함하여 랜덤하게 뽑을 수 있음)
    모든 데이터 출력
    검색 (음식점 이름, 분류, 걸리는 시간안에 있는 음식점, 일정 가격보다 낮은 메뉴가 있는 음식점)
    삭제 (폐업 시 삭제)
    프로그램 종료

In [2]:
#DTO
class RestInfo:
    def __init__(self,rest_name, category, topmenu, address, walking_time):
        self.rest_name = rest_name
        self.address = address
        self.walking_time = walking_time
        self.category = category
        self.topmenu = topmenu
    
    def setName(self, rest_name):
        self.rest_name = rest_name
    
    def getName(self):
        return self.rest_name

    def setMenu(self, topmenu):
        self.topmenu = topmenu
    
    def getMenu(self):
        return self.topmenu
    
    def setAdd(self, address):
        self.address = address
    
    def getAdd(self):
        return self.address

    def setWT(self, walking_time):
        self.walking_time = walking_time
    
    def getWT(self):
        return self.walking_time
    
    def setCG(self, category):
        self.category = category
    
    def getCG(self):
        return self.category

In [3]:
#DAO
import MySQLdb
class RestDao:
    def __init__(self):
        self.db = None
        
    def connect(self):
        self.db = MySQLdb.connect('localhost', 'root', '1234', 'kdt')
    
    def disconnect(self):
        self.db.close()
        
    def randomRest(self):
        self.connect()
        cur = self.db.cursor()
        sql = "select id from restaurant"
        cur.execute(sql)
        idlist = []# restaurant 테이블에 있는 모든 id를 불러와서 저장할 빈 리스트 생성
        while True:# 커서를 이동하면서 검색된 데이터를 한줄씩 불러와서 불러올 데이터가 없을때까지 idlist에 추가
            row = cur.fetchone()
            if row:
                idlist.append(row[0])
            else:
                break
        idset = set()#빈 세트를 생성

        while True:# 빈 세트에 idlist에 있는 id 값을 랜덤으로 추출하여 추가하되, 중복값 없이 3개가 채워질 때까지 반복
            temp = random.choice(idlist)
            idset.add(temp)
            if len(idset) == 3:
                break
        idsetlist = list(idset)# idset를 리스트로 변환
        randResult = []
        for i in idsetlist:# idsetlist에 저장된 3개의 id값과 일치하는 행을 한줄씩 가져와서 randResult 리스트에 추가
            sql2 = "select rest_name, category, topmenu, address, walking_time from restaurant where id = %s"
            data2 = (i,)
            cur.execute(sql2, data2)
            temp2 = cur.fetchone()
            randResult.append(temp2)
        self.disconnect()
        return randResult#랜덤으로 추출된 id값을 가진 행 3개를 가진 randResult리스트를 반환
    
    def searchName(self, rest_name):# 음식점 이름을 입력값으로 받는다
        self.connect()
        cur = self.db.cursor(MySQLdb.cursors.DictCursor)#출력 시 컬럼명을 key로 갖는 딕셔너리 형태로 결과를 받기 위함
        sql = "select rest_name, category, topmenu, address, walking_time from restaurant where rest_name = %s;"
        data = (rest_name, )# 입력받은 음식점 이름을 가진 행을 검색
        cur.execute(sql, data)
        result = cur.fetchall()#검색된 모든 행을 가져온다
        self.disconnect()
        return result# 검색된 모든 행을 튜플로 가지고있는 result변수를 반환
    
    def searchCT(self, category):# 음식점 분류를 입력값으로 받는다
        self.connect()
        cur = self.db.cursor(MySQLdb.cursors.DictCursor)
        sql = "select rest_name, category, topmenu, address, walking_time from restaurant where category = %s;"
        data = (category, )#입력받은 음식점 분류를 가진 행을 검색
        cur.execute(sql, data)
        result = cur.fetchall()#검색된 모든 행을 가져온다
        return result# 검색된 모든 행을 튜플로 가지고있는 result변수를 반환
        
    def searchWT(self, walking_time):# 음식점까지 도보로 걸리는 시간(분)을 입력값으로 받는다
        self.connect()
        cur = self.db.cursor(MySQLdb.cursors.DictCursor)
        sql = "select rest_name, category, topmenu, address, walking_time from restaurant where walking_time <= %s;"
        data = (walking_time, )# 입력받은 시간보다 적은 값을 가진 행을 검색
        cur.execute(sql, data)
        result = cur.fetchall()#검색된 모든 행을 가져온다
        return result# 검색된 모든 행을 튜플로 가지고있는 result변수를 반환
    
    def selectAll(self):#저장된 모든 데이터를 불러온다.
        self.connect()
        cur = self.db.cursor(MySQLdb.cursors.DictCursor)
        sql = "select rest_name, category, topmenu, address, walking_time from restaurant"
        cur.execute(sql)
        datasDB = cur.fetchall()#검색된 모든 행을 가져온다
        self.disconnect()
        return datasDB# 검색된 모든 행을 튜플로 가지고있는 datasDB변수를 반환
    
    def insert(self, info):#입력받은 정보를 DB에 저장한다
        self.connect()
        cur = self.db.cursor()   
        try:# 입력받은 값들 중, 음식점 이름은 중복되는 경우가 있기 때문에, 주소를 기준으로 중복값을 저장하지 않는다.
            sql = "select address, rest_name from restaurant where address = %s and rest_name = %s"
            data = (info.getAdd(),info.getName())
            result = cur.execute(sql, data)
            assert result == 0 #select로 검색한 값이 없다면 중복값이 없다는 것이므로, 입력받은 데이터를 DB에 추가
            sql2 = "insert into restaurant(rest_name, category, topmenu, address, walking_time) values(%s, %s, %s, %s, %s)"
            data2 = (info.getName(), info.getCG(), info.getMenu(), info.getAdd(), info.getWT())#DTO 역할의 클래스에서 입력값을 받아 return한 변수를 사용
            cur.execute(sql2, data2)
            self.db.commit()#데이터가 수정되었기 때문에 commit으로 변경사항을 저장한다.
            print('음식점이 등록되었습니다.')
        except Exception as e:
            print(e)
            print('이미 등록된 음식점 입니다.')
        self.disconnect()
        
    def delete(self, rest_name):#음식점 이름을 입력받음
        self.connect()
        cur = self.db.cursor(MySQLdb.cursors.DictCursor)
        sql = "select id, rest_name, category, topmenu, address, walking_time from restaurant where rest_name = %s"#입력받은 음식점 이름으로 해당되는 데이터를 검색
        data = (rest_name,)
        result = cur.execute(sql, data)
        if result == 0:#검색된 데이터가 없다면 저장된 데이터가 아니기 때문에 아래 문구를 출력
            print('삭제할 음식점 정보가 없습니다.')
        else:
            idList=[]
            row = cur.fetchall()
            for i in row:# 입력받은 음식점 이름이 여러개인 경우 고유한 id값을 포함하여 정보를 출력
                print(f"id 번호: {i['id']} ,이름: {i['rest_name']}, 분류: {i['category']}, 대표메뉴: {i['topmenu']}, 주소: {i['address']}, 소요시간(도보): {i['walking_time']}분")
                idList.append(i['id'])# idList에 검색된 id값만 추가 
            idNum = input('삭제할 음식점의 id 번호를 입력하세요: ')# 여러 음식점이 검색되는 경우 삭제할 음식점의 고유 번호인 id를 입력
            sql2 = "delete from restaurant where id = %s"
            data2 = (idNum, )
            cur.execute(sql2, data2)
            self.db.commit()
            if int(idNum) in idList:# 검색된 id값이 아닌 다른 id 값을 입력했을 경우 id 번호가 잘못 입력됨을 알려주고, 올바르게 입력했을 경우 완료됨을 알려줌
                print('삭제가 완료되었습니다.')
            else:
                print('id 번호를 잘못 입력하였습니다.')
        self.disconnect()
        

In [4]:
#VO
import random
class RestService:
    def __init__(self):
        self.dao = RestDao()
    
    def randLunch(self):
        result = self.dao.randomRest()
        for i in result:# 랜덤으로 출력된 3개의 튜플을 보기좋게 출력
            print(f"이름: {i[0]}, 분류: {i[1]}, 대표메뉴: {i[2]}, 주소: {i[3]}, 소요시간(도보): {i[4]}분")
        
    def searchRest(self):# 검색할 조건을 입력받기 위한 메서드
        condition = input('검색 조건을 선택하세요.\n1. 음식점 이름 / 2. 음식점 분류 / 3. 소요 시간(도보)\n 번호를 입력하세요.: ')
        while True:
            try:
                assert int(condition) in range(1,5)
                if condition == '1':
                    inputname = input('음식점 이름을 입력하세요: ')
                    nameResult = self.dao.searchName(inputname.strip())#검색된 값의 양쪽 공백을 모두 없앤다
                    if len(nameResult) == 0:# searchName메서드에서 반환 받은 값이 없는 경우 해당되는 데이터가 없음을 알려줌
                        print('찾으시는 정보가 없습니다.')
                    for i in nameResult:# searchName메서드에서 반환된 값을 보기좋게 출력
                        print(f"이름: {i['rest_name']}, 분류: {i['category']}, 대표메뉴: {i['topmenu']}, 주소: {i['address']}, 소요시간(도보): {i['walking_time']}분")
                elif condition == '2':
                    inputCT = input('음식점 분류명을 입력하세요(한식, 중식, 일식, 분식, 양식, 동남아, 다이어트, 기타): ')
                    CTresult = self.dao.searchCT(inputCT.strip())
                    if len(CTresult) == 0:
                        print('찾으시는 정보가 없습니다.')
                    for j in CTresult:
                        print(f"이름: {j['rest_name']}, 분류: {j['category']}, 대표메뉴: {j['topmenu']}, 주소: {j['address']}, 소요시간(도보): {j['walking_time']}분")
                elif condition == '3':
                    inputWT = input('소요시간(도보)을 입력하세요(분): ')
                    WTresult = self.dao.searchWT(inputWT)
                    if len(WTresult) == 0:
                        print('찾으시는 정보가 없습니다.')
                    for z in WTresult:
                        print(f"이름: {z['rest_name']}, 분류: {z['category']}, 대표메뉴: {z['topmenu']}, 주소: {z['address']}, 소요시간(도보): {z['walking_time']}분")
                break
            except Exception as e:
                print(e)
                print('다시 입력하세요.')
                break
    def printAll(self):# selectAll메서드에서 반환된 값을 보기좋게 출력
        datas =  self.dao.selectAll()
        for i in datas:
            print(f"이름: {i['rest_name']}, 분류: {i['category']}, 대표메뉴: {i['topmenu']}, 주소: {i['address']}, 소요시간(도보): {i['walking_time']}분")
    def insertInfo(self):# 저장할 음식점 정보를 필드별로 입력받음
        rest_name = input('음식점 이름을 입력하세요: ')
        category = input('음식점 분류를 입력하세요(한식, 중식, 일식, 분식, 양식, 동남아, 다이어트, 기타): ')
        topmenu = input('음식점의 대표 메뉴를 입력하세요: ')
        address = input('음식점 주소를 입력하세요(층 또는 호수까지 포함, 도로명 주소): ')
        walking_time = input('학원에서 음식점까지 도보로 걸리는 시간을 입력하세요(분): ')
        info = RestInfo(rest_name.strip(), category.strip(), topmenu.strip(), address.strip(), walking_time)# RestInfo 객체를 생성
        self.dao.insert(info)# insert 메서츠 호출
        
    def deleteInfo(self):#음식점 삭제 시 음식점 이름을 입력받는다.
        rest_name = input('삭제할 음식점 이름을 입력하세요: ')
        self.dao.delete(rest_name)
        

In [5]:
class Menu:
    def __init__(self):
        self.service = RestService()
        
    def run(self):
        while True:
            try:
                print()
                print('수행할 작업을 선택하세요')
                inputMenu = input('1. 오늘의 점심 추천\n2. 음식점 검색\n3. 모든 음식점 출력\n4. 음식점 등록\n5. 음식점 삭제\n6. 프로그램 종료\n 번호 입력: ')
                if inputMenu == '1':
                    self.service.randLunch()
                elif inputMenu == '2':
                    self.service.searchRest()
                elif inputMenu == '3':
                    self.service.printAll()
                elif inputMenu == '4':
                    self.service.insertInfo()
                elif inputMenu =='5':
                    self.service.deleteInfo()
                elif inputMenu == "6":
                    print('프로그램을 종료합니다.')
                    break
            except Exception as e:
                print(e)
                print('다시 입력하세요!')

In [6]:
test = Menu()

In [ ]:
test.run()


수행할 작업을 선택하세요
1. 오늘의 점심 추천
2. 음식점 검색
3. 모든 음식점 출력
4. 음식점 등록
5. 음식점 삭제
6. 프로그램 종료
 번호 입력: 2
검색 조건을 선택하세요.
1. 음식점 이름 / 2. 음식점 분류 / 3. 소요 시간(도보)
 번호를 입력하세요.: 2
음식점 분류명을 입력하세요(한식, 중식, 일식, 분식, 양식, 동남아, 다이어트, 기타): 일식
이름: 탄, 분류: 일식, 대표메뉴: 돈까쓰, 주소: 서울 강남구 테헤란로20길 13 덕성빌딩 2층, 소요시간(도보): 3분
이름: 호센바, 분류: 일식, 대표메뉴: 어묵우동(단일메뉴), 주소: 서울 강남구 논현로95길 29-8 1층, 소요시간(도보): 6분

수행할 작업을 선택하세요
